In [2]:
import sys
import pickle
import scipy
from scipy import signal
from scipy import stats
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import cohen_kappa_score

import math
from collections import OrderedDict


import matplotlib.pyplot as plt

sys.path.append('D:\Diamond\code')
from csp_james_2 import *

sys.path.append('D:\Diamond\code')
from thesis_funcs_19_03 import *

import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.autograd import Variable

from torch.optim import lr_scheduler

import csv
from random import randint
import random
import datetime

In [18]:
to_save = 0

In [15]:
dir_raw_data = 'E:\\Diamond\\bci_iv\\DATA\\2a\\matlab_raw\\'

for subject in range (1,3):
    for ses in ['T','E']:
        sub = 'A0' + str(subject) + ses
        fileroot = dir_raw_data + sub + '.mat'

        raw_mat_data = scipy.io.loadmat(fileroot)['data']


        #ind = 7 #0 = raw data, 1=start ind, 2=lables, 3 = fs, 4 = labels meaning, 5=artifects, 6=sex, 7=age
        num_ch = 22
        num_seconds = 7 #startof trial to 1s after end of MI
        virgin = 1

        num_runs = np.shape(raw_mat_data)[1]
        for tr in range (0,num_runs): #actually "tr" should be "run", represents the 6 runs of recordings, each run has 48 trials
            if any(raw_mat_data[0][tr][0][0][1] ):
                #for each run (total of 7 or 6), extract eeg and labels
                #raw_eeg, raw recording, all the trails form all channels
                raw_eeg = raw_mat_data[0][tr][0][0][0]
                #sampling frequency, 250Hz
                fs = float(raw_mat_data[0][tr][0][0][3])

                #index of trails without artefects
                good_ind = np.where(raw_mat_data[0][tr][0][0][5] == 0)[0]

                #trail start time(ind) position for the good trials
                start_ind = raw_mat_data[0][tr][0][0][1].squeeze()
                start_ind = start_ind[good_ind]

                #good trials labels
                LABELS = raw_mat_data[0][tr][0][0][2].squeeze()
                LABELS = LABELS[good_ind]

                #initiating matrix to store sepereated eeg trails, for the 22 eeg channels
                EEG_raw = np.zeros((len(good_ind), num_ch, int(fs*num_seconds)))

                #for eahc good trail, extract it from raw recording and store into EEG_raw matrix, in shape num_trials X num_chanels X num_time
                for i in range (0, len(good_ind)):
                    EEG_raw[i] = np.transpose(raw_eeg[start_ind[i]:start_ind[i]+int(7*fs), 0:22])

                if virgin:
                    EEG_RAW = EEG_raw
                    LABELS_RAW = LABELS
                    virgin = 0
                elif virgin == 0:
                    EEG_RAW = np.concatenate((EEG_RAW, EEG_raw), axis = 0)
                    LABELS_RAW = np.concatenate((LABELS_RAW, LABELS), axis = 0)
            else:
                print ('run ' + str(tr) + ' is empty')


        
        if to_save == 1:
            pickle.dump( EEG_RAW, open('E:\\Diamond\\bci_iv\\DATA\\2a\\extract_raw\\' + sub + ".pickle", "wb" ) )
            pickle.dump( LABELS_RAW, open('E:\\Diamond\\bci_iv\\DATA\\2a\\extract_raw\\' + sub + "_LABELS.pickle", "wb" ) )

run 0 is empty
run 1 is empty
run 2 is empty
run 0 is empty
run 1 is empty
run 2 is empty
run 0 is empty
run 1 is empty
run 2 is empty
run 0 is empty
run 1 is empty
run 2 is empty


In [17]:
np.shape(EEG_RAW), np.shape(LABELS_RAW)

((283, 22, 1750), (283,))